# SPARK RDD part 1

In [ ]:
import pyspark
sc = pyspark.SparkContext(appName="RDDstart")

### Tworzenie RDD

#### Lokalna kolekcja

W realnych zastosowaniach RDD tworzy się poprzez wczytanie danych z rozproszonych zasobów sieciowych takich jak na przykład HDFS, Cassandra, MongoDB, Amazon S3 i wielu innych. Mając doczynienia z dużo mniejszymi zbiorami, możemy je zaczytać w programie sterownika i przekształcić programowo w rozproszony zbiór RDD. Pomoże nam w tym funkcja parallelize. 

In [ ]:
RDDlist = sc.parallelize(list(range(10)))

In [ ]:
RDDlist

In [ ]:
RDDlist.collect()

In [ ]:
RDDlist.take(5)

**_UWAGA: `collect` i `take` to niebezpieczne operacje - mogą doprowadzić do zapchania drivera i przerwania działania aplikacji_**

#### Plik

In [ ]:
RDDfile = sc.textFile(".../RDD.txt")
RDDfile

In [ ]:
RDDfile.collect()

In [ ]:
RDDfile.take(5)

#### Inne RDD

In [ ]:
RDDother = RDDfile.flatMap(lambda x: x.split(","))

In [ ]:
RDDother.collect()

In [ ]:
RDDother.take(5)

### Transformacje

##### map(fun)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD

In [ ]:
RDDmap1 = RDDother.map(lambda x: int(x))
#Powyższe transformacje "wykonały się", ale nic się jeszcze nie stało. Nie dostaliśmy żadnego wyniku.
#Dzieje się tak ponieważ transformacje są leniwe. Dzięki temu można stworzyć cały graf przetwarzania przed 
#jego uruchomieniem.
RDDmap1.collect()

In [ ]:
RDDmap1.map(lambda x: x+1).collect()

In [ ]:
RDDmap1.map(lambda x: (x,1)).collect()

In [ ]:
RDDlist2 = sc.parallelize([list(range(5)),list(range(5,10))])

In [ ]:
RDDlist2.collect()

In [ ]:
RDDlist2.map(sum).collect()

In [ ]:
RDDlist2.map(lambda x: [y + 1 for y in x]).collect()

> **TODO**: Oblicz sumę całkowitą RDDlist2

##### flatMap(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD oraz spłaszczeniu rezultatu

In [ ]:
RDDlist2.flatMap(lambda x: x).collect()

In [ ]:
RDDlist2.flatMap(lambda x: x*2).collect()

In [ ]:
RDDlist2.flatMap(lambda x: [y*2 for y in x]).collect()

In [ ]:
RDDlist3 = sc.parallelize([[[[0],[1]],[[2],[3]]],[[[4],[5]],[[6],[7]]]])
RDDlist3.collect()

In [ ]:
RDDlist3.flatMap(lambda x: x).collect()

In [ ]:
RDDlist3.flatMap(lambda x: x).flatMap(lambda x: x).collect()

In [ ]:
RDDlist3.flatMap(lambda x: x).flatMap(lambda x: x).flatMap(lambda x: x).collect()

> **TODO**: Osiągnij taki sam wynik jak w komórce powyżej używając na RDDlist3: 1 x map i 2 x flatMap

##### mapValues(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdej wartości oryginalnego RDD zawierającego pary klucz-wartość

In [ ]:
RDDpair = sc.parallelize([("A",["Adam","Ada","Adrian"]),("B",["Bonifacy","Barnaba"])])
RDDpair.collect()

In [ ]:
RDDpair.mapValues(lambda val: len(val)).collect()

In [ ]:
RDDpair.mapValues(lambda val: len(val)).mapValues(lambda val: val*2).collect()

In [ ]:
RDDpair.mapValues(lambda val: len(val)*2).collect()

Te same rezultaty można osiągnąć używając `map`, jednak jest to mniej wygodne

In [ ]:
RDDpair.map(lambda x: (x[0],len(x[1]))).collect()

In [ ]:
RDDpair.map(lambda x: (x[0],len(x[1]))).map(lambda x: (x[0],x[1]*2)).collect()

In [ ]:
RDDpair.map(lambda x: (x[0],len(x[1]*2))).collect()

> **TODO**: Zmodyfikuj wartości w RDDpair tak aby zawierały nie imiona a liczby liter w imionach, następnie zsumuj liczby liter, wykonaj zadanie w dwóch wariantach - używając tylko `mapValues` i tylko `map`

##### flatMapValues(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdej wartości oryginalnego RDD zawierającego pary klucz-wartość oraz spłaszczeniu rezultatu

In [ ]:
RDDpair.flatMapValues(lambda x: x).collect()

In [ ]:
RDDpair.flatMapValues(lambda x: [y.lower() for y in x]).collect()

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (litera, (imię, l. liter))

##### keys(), values()

Metody te tworzą nowe RDD odpowiednio z kluczy i wartości oryginalnego RDD (klucz, wartość)

In [ ]:
RDDpair.values().collect()

In [ ]:
RDDpair.keys().collect()

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (imię, l. liter)

##### filter(func)

Zwraca nowe RDD zawierające jedynie elementy które spełniają predykat

In [ ]:
RDDmap1.filter(lambda x: x > 3).collect()

In [ ]:
RDDpair.filter(lambda x: x[0] == "A").collect()

Dobrą praktyką optymalizującą działanie programu jest filtrowanie RDD możliwie jak najwcześniej. Z poniższych dwóch komórek z kodem to druga jest bardziej optymalna.

In [ ]:
RDDpair.flatMapValues(lambda x: x).filter(lambda x: x[0] != "B").collect()

In [ ]:
RDDpair.filter(lambda x: x[0] != "B").flatMapValues(lambda x: x).collect()

> **TODO**: Odfiltruj parzyste liczby z RDDmap1

##### join(RDD)

Zwraca RDD zawierające pary elementów z identycznymi kluczami w łączonych RDD.

In [ ]:
RDDpair.join(RDDpair.flatMapValues(lambda x: x)).collect()

In [ ]:
RDDpair.collect()

In [ ]:
RDDpair.flatMapValues(lambda x: x).collect()

##### union(RDD)

Zwraca RDD wynikłe z połączenia dwóch RDD

In [ ]:
RDDmap1.union(RDDlist).collect()

> **TODO**: Połącz dwa RDD powstałe na podstawie RDDpair o strukturze: (imię, l. liter) oraz (litera, imię), w wynikowym RDD mają znaleźć się jedynie obiekty których klucz zaczyna się na "B"

##### distinct()

Zwraca RDD zawierające jedynie unikalne wartości z oryginalnego RDD

In [ ]:
RDDmap1.union(RDDlist).collect()

In [ ]:
RDDmap1.union(RDDlist).distinct().collect()

##### groupBy(func)

Zwraca RDD z pogrupowanymi elementami

In [ ]:
RDDlist.groupBy(lambda x: x % 2).collect()

In [ ]:
RDDlist.groupBy(lambda x: x % 2).map(lambda x: (x[0], list(x[1]))).collect()

In [ ]:
RDDlist.groupBy(lambda x: x % 2).map(lambda x: (x[0], [y*3 for y in x[1]])).collect()

In [ ]:
RDDpair2 = sc.parallelize([('Adam', 4), ('Ada', 3), ('Adrian', 6), ('Bonifacy', 8), ('Barnaba', 7)])

In [ ]:
RDDpair2.groupBy(lambda x: "l.l.>4" if x[1] > 4 else "l.l.<=4").map(lambda x: (x[0], list(x[1]))).collect()

> **TODO**: Pogrupuj RDDpair2 ze względu na pierwszą literę imienia

##### groupByKey()

Zwraca RDD z wartościami pogrupowanymi w pojedynczą sekwencję dla każdego klucza. (_Jeśli grupowanie wykonywane jest w celu przeprowadzenia agregacji dla każdego klucza, optymalniejsze będzie wykorzystanie `reduceByKey` lub `aggregateByKey`._)

In [ ]:
RDDpair3 = sc.parallelize([('A', 'Adam'),('A', 'Ada'),('A', 'Adrian'),('B', 'Bonifacy'),('B', 'Barnaba')])

In [ ]:
RDDpair3.groupByKey().collect()

In [ ]:
RDDpair3.groupByKey().mapValues(list).collect()

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (litera, (imię, l. liter)), następnie pogrupuj je po literze (zamień nowe wartości na listę tak aby były czytelne po użyciu `collect`)

##### reduceByKey(func)

Zwraca RDD z połączonymi wartościami dla każdego klucza. Funkcja redukująca musi być asocjacyjna _[(a x b) x c = a x (b x c)]_ i przemienna _[a x b = b x a]_. 

In [ ]:
RDDpair4 = RDDpair3.mapValues(len)
RDDpair4.collect()

In [ ]:
RDDpair4.reduceByKey(lambda x,y: x+y).collect()

> **TODO**: Uzyskaj iloczyn dla każdego klucza w RDDpair4

##### aggregateByKey(zeroValue, seqFunc, combFunc)

Rozbudowana wersja `reduceByKey` pozwalająca na zwrócenie wartości o innym typie niż oryginalne. Konieczne jest podanie trzech parametrów:
- zeroValue - domyślna wartość neutralna dla agregacji (dodawanie: 0, mnożenie: 1, tworzenie zbioru unikatowych wartości: pusty zbiór, itd.),
- seqFunc - funkcja agregująca wartości w oryginalnym RDD, przyjmuje dwa parametry, gdzie drugi jest włączany (dodawany itp.) do pierwszego
- combFunc - funkcja łącząca wartości uzyskane z pierwszej funkcji dla kluczy

In [ ]:
RDDpair3.collect()

In [ ]:
RDDpair3.aggregateByKey(0, (lambda acc,x: acc+len(x)), (lambda acc1,acc2: acc1+acc2)).collect()

In [ ]:
RDDpair3.glom().collect()

Co się wydarzyło powyżej?

|RDDpair3 | seqFunc | seqFunc out. | combFunc | combFunc out. |
|:--------|:--------|:-------------|:---------|:------------- |
|('A', 'Adam') => | ('A', 0 + len('Adam')) => | ('A', 4) |  |  |
|('A', 'Ada') => | ('A', 0 + len('Ada')) => | ('A', 3) |  |  |
|('A', 'Adrian') => | ('A', 0 + len('Adrian')) => | ('A', 6) => | ('A', 4 + 3 + 6) => | ('A', 13) |
|('B', 'Bonifacy') => | ('B', 0 + len('Bonifacy')) => | ('B', 8) => | ('B', 8 + 7) => | ('B', 15) |
|('B', 'Barnaba') => | ('B', 0 + len('Barnaba')) => | ('B', 7) |  |  |

In [ ]:
RDDpair3.aggregateByKey((0.,0.), (lambda acc,x: (acc[0]+len(x),acc[1]+1)), \
                        (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1]))).collect()

In [ ]:
RDDpair3.aggregateByKey((0.,0.), (lambda acc,x: (acc[0]+len(x),acc[1]+1)), \
                        (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1]))).mapValues(lambda x: x[0]/x[1]).collect()

> **TODO**: Z RDDpair3 uzyskaj RDD (klucz, iloczyn długości imion)

### Akcje

##### collect()

Zwraca elementy zbioru na driver.

In [ ]:
RDDlist.collect()

##### collectAsMap()

Zwraca RDD (K, V) jako słownik.

In [ ]:
RDDpair2.collectAsMap()

##### take(n)

Zwraca `n` pierwszych elementów zbioru na driver.

In [ ]:
RDDlist.take(2)

##### takeSample(withReplacement, num)

Zwraca losową próbę `num` elementów ze zwracaniem lub bez.

In [ ]:
RDDlist.takeSample(True,12)

In [ ]:
RDDlist.takeSample(False, 5)

##### takeOrdered(n, [key])

Zwraca `n` pierwszych elementów zbioru stosując naturalny porządek lub inny wskazany.

In [ ]:
RDDlist.takeOrdered(5)

In [ ]:
RDDlist.takeOrdered(5, (lambda x: -x))

##### first()

Zwraca pierwszy element zbioru. Podobne do `take(1)`.

In [ ]:
RDDlist.first()

##### count()

Zwraca liczbę elementów w zbiorze.

In [ ]:
RDDlist.count()

##### sum()

Zwraca sumę elementów w RDD

In [ ]:
RDDlist.sum()

##### countByKey()

Dla RDD (K, V) zwraca słownik (hashmap) typu (K, Int) z liczbą wystąpień kluczy.

In [ ]:
RDDpair4.countByKey()

In [ ]:
RDDpair4.countByKey()["A"]

##### saveAsTextFile(path)

Zapisuje elementy zbioru do pliku (plików) tekstowych w podanym katalogu. Spark wywoła metodę `toString` na każdym elemencie RDD aby przekształcić je na linijkę teksu w pliku.

In [ ]:
RDDlist.saveAsTextFile("...")

##### reduce(func)

Agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a].

In [ ]:
RDDlist.reduce(lambda x,y: x+y)

In [ ]:
RDDlist2.reduce(lambda x,y: x+y)

##### aggregate(zeroValue, seqOp, combOp)

Działa podobnie do `aggregateByKey`

In [ ]:
RDDother.aggregate(0, (lambda acc,x: acc + int(x)), (lambda acc1,acc2: acc1 + acc2))

> **TODO**: Po odfiltrowaniu "0" oblicz iloczyn wartości RDDother

##### fold(zeroValue, op)

Zwraca zagregowane przy pomocy funkcji `op` wartości RDD. Funkcja `op` przyjmuje 2 wartości i musi być asocjacyjna _[(a x b) x c = a x (b x c)]_.

In [ ]:
RDDlist.fold(0, (lambda acc,x: acc + x))

> **TODO**: Po odfiltrowaniu 0 i liczb nieparzystych oblicz iloczyn wartości RDDlist

##### max(), mean(), min(), stdev(), variance(), stats()

In [ ]:
RDDlist.max()

In [ ]:
RDDlist.mean()

In [ ]:
RDDlist.min()

In [ ]:
RDDlist.stdev()

In [ ]:
RDDlist.variance()

In [ ]:
RDDlist.stats()

##### reduceByKeyLocally(func)

Działa analogicznie do `reduceByKey` z tym, że zwraca wynik do drivera. 

In [ ]:
RDDpair4.reduceByKeyLocally(lambda x,y: x+y)